In [3]:
from sage.all import *
import hashlib
from random import randint

In [4]:
# Define secp256k1 curve
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
F = FiniteField(p)
a = 0
b = 7
E = EllipticCurve(F, [a, b])

# Define Bitcoin's chosen secp256k1 generator
G_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
G = E.lift_x(G_x)  # Lift x-coordinate to get full generator point

# Define the order of the group
N = G.order()

In [5]:
# --- Implementing Schnorr Signature Scheme ---
def schnorr_prove(secret, G):
    k = randint(1, N - 1)  # Random nonce
    R = k * G  # Commitment
    return R, k

def generate_challenge(R, public_key):
    challenge = int(hashlib.sha256(str(R.xy() + public_key.xy()).encode()).hexdigest(), 16) % N
    return challenge

def schnorr_respond(secret, k, challenge):
    response = (k + challenge * secret) % N
    return response

def schnorr_verify(G, R, challenge, response, public_key):
    return response * G == R + challenge * public_key

In [4]:
secret = randint(1, N - 1)  # Private key
public_key = secret * G  # Public key

# Prover step: generates commitment R
R, k = schnorr_prove(secret, G)

# Verifier step: generates challenge e
challenge = generate_challenge(R, public_key)

# Prover step: responds with s
response = schnorr_respond(secret, k, challenge)

# Verifier step: verifies the proof
assert schnorr_verify(G, R, challenge, response, public_key)

print("Schnorr Proof verified successfully!")

Schnorr Proof verified successfully!
